In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential
import os

In [ ]:
train_path="your train dataset path"
test_path="your test dataset path"

In [ ]:
class VisualImage:
    def __init__(self, path):
        self.path = path
    
    def VisualImageMale(self):
        male_path = os.path.join(self.path,"male")
        male_images = sorted(os.listdir(male_path))[:12]
        rows = 4
        cols = 3
        fig,axs = plt.subplots(rows,cols,figsize=(6,6))
        for i in range(rows):
            for j in range(cols):
                img_path = os.path.join(male_path, male_images[i*cols + j])
                img = cv2.imread(img_path)
                # Resmi subplot'a ekliyoruz
                axs[i, j].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                axs[i, j].axis('off')
        plt.show()
    
    def VisualImageFemale(self):
        female_path = os.path.join(self.path,"female")
        female_images = sorted(os.listdir(female_path))[:12]
        rows = 4
        cols = 3
        fig,axs = plt.subplots(rows,cols,figsize=(6,6))
        for i in range(rows):
            for j in range(cols):
                img_path = os.path.join(female_path, female_images[i*cols + j])
                img = cv2.imread(img_path)
                # Resmi subplot'a ekliyoruz
                axs[i, j].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                axs[i, j].axis('off')
        plt.show()
        
        

In [ ]:
image_visualizer = VisualImage(train_path) # you can see the test_path image_visualizer = VisualImage(test_path)  
image_visualizer.VisualImageFemale()
image_visualizer.VisualImageMale()

In [ ]:
EPOCH = 2
BATCH_SIZE = 32
class_names = ["female","male"]

In [ ]:
x_train = ImageDataGenerator(rescale = 1./255)
x_test = ImageDataGenerator(rescale = 1./255)

train_gen = x_train.flow_from_directory(
    directory = train_path,
    batch_size = BATCH_SIZE,
    classes = class_names,
    class_mode = 'categorical',
    target_size = (220,220),
    shuffle = True
)

test_gen = x_test.flow_from_directory(
    directory = test_path,
    batch_size = BATCH_SIZE,
    classes = class_names,
    class_mode = 'categorical',
    target_size = (220,220),
    shuffle = True
)

In [ ]:

class MyCNNModel:
    def __init__(self, input_shape=(220, 220, 3)):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
        self.model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        self.model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.2))
        self.model.add(Flatten())
        self.model.add(Dense(100, activation='relu'))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.2))
        self.model.add(Dense(2, activation='softmax'))
        self.history = None

    def compile_model(self, optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']):
        self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        
    
    def summary(self):
        self.model.summary()
        
        
    def train_model(self):
        self.history = self.model.fit(train_gen,
                                epochs = EPOCH,
                                validation_data = test_gen)
        
    def plot_graph(self):
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['val_accuracy'])
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.legend(['train','test'],loc='best')       



my_model = MyCNNModel()
my_model.compile_model()
my_model.summary()
my_model.train_model()
my_model.plot_graph()

In [ ]:
my_model.save("save location")

In [ ]:
face_classifier=cv2.CascadeClassifier("face_classifier location")

In [ ]:
class predictReelTime:
    def __init__(self,model,face_classifier):
        self.model = model
        self.face_classifier = face_classifier
        self.cap = None
    def predict(self,cap):
        self.cap = cap
        prediction=self.model.model.predict(self.cap)
        th = 0.8
        class_id = np.argmax(prediction)
        if (prediction[0][class_id] > th):
            return class_id
   
    def reelTimePredict(self):
        vid=cv2.VideoCapture(0)
        if not vid.isOpened():
            print('cannot open the camera')
            exit()
        while True:
            ret,cap=vid.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            gray=cv2.cvtColor(cap,cv2.COLOR_BGR2GRAY)
            face=self.face_classifier.detectMultiScale(gray, 1.05, 5)
            if face is ():
                print('no face detected')

            for (x,y,a,b) in face:
                cv2.rectangle(cap,(x,y),(x+a,y+b),(127,0,255),2)
                roi_color=cap[y:y+b,x:x+a]
                img = cv2.resize(roi_color, (220,220), interpolation=cv2.INTER_CUBIC)
                img = img.astype('float16')
                img = np.expand_dims(img, axis=0)
                score= self.predict(img)

                if score==1:
                    cv2.putText(cap,'Male',(x+a,y+b), cv2.FONT_HERSHEY_SIMPLEX ,0.5,(255,0,0),1, cv2.LINE_AA)
                if score==0:
                    cv2.putText(cap,'Female',(x+a,y+b), cv2.FONT_HERSHEY_SIMPLEX ,0.5,(255,0,0),1, cv2.LINE_AA)  
                cv2.imshow('image',cap)
                cv2.waitKey(2)
            if cv2.waitKey(2) & 0xFF==ord('q'):
                break
        vid.release()
        cv2.destroyAllWindows()

prediction = predictReelTime(model = my_model,face_classifier=face_classifier)
prediction.reelTimePredict()